In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [14]:
import gensim
from gensim import corpora
import re, string
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.style.use('bmh')

In [67]:
#path = 'file/'
# path = './'
business = pd.read_json('./yelp_academic_dataset_business.json', lines=True)
# business_hour = pd.read_csv(path + 'yelp_business_hours.csv')
# checkin = pd.read_csv(path + 'yelp_checkin.csv')
# tips = pd.read_csv(path+'yelp_tip.csv',nrows = 10000)
# review = pd.read_csv(path+'yelp_review.csv',nrows = 10000)
# Code


In [68]:
business.describe()

,is_open,latitude,longitude,review_count,stars
count,192609.000000,192609.000000,192609.000000,192609.000000,192609.000000
mean,0.823040,38.541803,-97.594785,33.538962,3.585627
std,0.381635,4.941964,16.697725,110.135224,1.018458
min,0.000000,33.204642,-115.493471,3.000000,1.000000
25%,1.000000,33.637408,-112.274677,4.000000,3.000000
50%,1.000000,36.144815,-111.759323,9.000000,3.500000
75%,1.000000,43.602989,-79.983614,25.000000,4.500000
max,1.000000,51.299943,-72.911982,8348.000000,5.000000


In [113]:
restaurants = business[business["categories"].str.contains("Restaurants", na=False)]

In [133]:
restaurants["categories"].str.contains("Restaurants", na=False).describe()

count     59371
unique        1
top        True
freq      59371
Name: categories, dtype: object

In [134]:
restaurants.describe()

,is_open,latitude,longitude,review_count,stars
count,59371.000000,59371.000000,59371.000000,59371.000000,59371.000000
mean,0.711408,40.121872,-92.366585,68.456452,3.441790
std,0.453112,5.002571,16.552078,176.636132,0.803304
min,0.000000,33.210781,-115.452994,3.000000,1.000000
25%,0.000000,35.262902,-112.064753,7.000000,3.000000
50%,1.000000,41.306488,-81.359873,21.000000,3.500000
75%,1.000000,43.707870,-79.427944,62.000000,4.000000
max,1.000000,51.294184,-72.934296,8348.000000,5.000000


In [176]:
def find_categories(business):
    categories = {}
    cols = business.columns
    for t in business.iterrows():
        for k,v in zip(cols,t[1]):
            if k == "categories":
                if v is not None:
                    for i in v.split(","):
                        s = i.strip()
                        if s in categories:
                            categories[s]= categories[s]+1
                        else:
                            categories[s]=1
    return categories


In [177]:
cat = find_categories(restaurants)

In [180]:
cat

{'Specialty Food': 2091,
 'Restaurants': 59371,
 'Dim Sum': 302,
 'Imported Food': 299,
 'Food': 14800,
 'Chinese': 4428,
 'Ethnic Food': 867,
 'Seafood': 2508,
 'Sushi Bars': 2258,
 'Japanese': 2716,
 'Breakfast & Brunch': 5381,
 'Mexican': 4618,
 'Tacos': 271,
 'Tex-Mex': 737,
 'Fast Food': 7257,
 'Italian': 4716,
 'Pizza': 6804,
 'Chicken Wings': 2705,
 'Sandwiches': 7332,
 'Salad': 2531,
 'Burgers': 5404,
 'Comfort Food': 860,
 'Patisserie/Cake Shop': 91,
 'Bars': 8182,
 'Nightlife': 8562,
 'Juice Bars & Smoothies': 908,
 'Vegan': 848,
 'Tapas Bars': 417,
 'Southern': 536,
 'Coffee & Tea': 3647,
 'American (Traditional)': 7107,
 'American (New)': 4882,
 'Steakhouses': 1603,
 'Modern European': 216,
 'Wine Bars': 956,
 'Hookah Bars': 147,
 'Persian/Iranian': 185,
 'Country Dance Halls': 11,
 'Middle Eastern': 1317,
 'Beer': 1135,
 'Wine & Spirits': 1135,
 'Lounges': 908,
 'Delis': 1955,
 'Fish & Chips': 319,
 'Polish': 80,
 'Korean': 923,
 'Ice Cream & Frozen Yogurt': 993,
 'Dessert

In [137]:
def find_attributes(data):
    biz_attributes = {}
    for i in data['attributes']:
        if i is not None:
            for e in i.keys():
                if e in biz_attributes:
                    biz_attributes[e]= biz_attributes[e]+1
                else:
                    biz_attributes[e]=1
    return biz_attributes

    

In [144]:
rest_attr = find_attributes(restaurants)
biz_attr = find_attributes(business)






In [150]:
rest_attr

{'RestaurantsReservations': 50411,
 'GoodForMeal': 29657,
 'BusinessParking': 47685,
 'Caters': 34908,
 'NoiseLevel': 41046,
 'RestaurantsTableService': 17153,
 'RestaurantsTakeOut': 52312,
 'RestaurantsPriceRange2': 52091,
 'OutdoorSeating': 48322,
 'BikeParking': 38886,
 'Ambience': 44909,
 'HasTV': 44854,
 'WiFi': 40709,
 'GoodForKids': 50661,
 'Alcohol': 44322,
 'RestaurantsAttire': 48540,
 'RestaurantsGoodForGroups': 50963,
 'RestaurantsDelivery': 49812,
 'BusinessAcceptsCreditCards': 34374,
 'BYOBCorkage': 1421,
 'DriveThru': 3191,
 'HappyHour': 3810,
 'Smoking': 2447,
 'WheelchairAccessible': 7913,
 'DogsAllowed': 4179,
 'CoatCheck': 2613,
 'Music': 3776,
 'BestNights': 2686,
 'BusinessAcceptsBitcoin': 3369,
 'GoodForDancing': 3534,
 'Corkage': 662,
 'BYOB': 28,
 'ByAppointmentOnly': 643,
 'AgesAllowed': 33,
 'AcceptsInsurance': 15,
 'DietaryRestrictions': 58,
 'HairSpecializesIn': 5,
 'Open24Hours': 7,
 'RestaurantsCounterService': 11}

In [42]:
tips = pd.read_json('./yelp_academic_dataset_tip.json', lines=True)




In [45]:
tips

,business_id,compliment_count,date,text,user_id
0,VaKXUpmWTTWDKbpJ3aQdMw,0,2014-03-27 03:51:24,"Great for watching games, ufc, and whatever el...",UPw5DWs_b-e2JRBS-t37Ag
1,OPiPeoJiv92rENwbq76orA,0,2013-05-25 06:00:56,Happy Hour 2-4 daily with 1/2 price drinks and...,Ocha4kZBHb4JK0lOWvE0sg
2,5KheTjYPu1HcQzQFtm4_vw,0,2011-12-26 01:46:17,Good chips and salsa. Loud at times. Good serv...,jRyO2V1pA4CdVVqCIOPc1Q
3,TkoyGi8J7YFjA6SbaRzrxg,0,2014-03-23 21:32:49,The setting and decoration here is amazing. Co...,FuTJWFYm4UKqewaosss1KA
4,AkL6Ous6A1atZejfZXn1Bg,0,2012-10-06 00:19:27,Molly is definately taking a picture with Sant...,LUlKtaM3nXd-E4N4uOk_fQ
5,kj1p6NwUGROOoqNySf0I1Q,0,2011-12-31 18:02:42,It's true! The drunken noodles are outrageous!,iAd8XaHxv05iXyEiZ62Ibw
6,0hudPyuCBlKg79OwKBw-eQ,0,2012-10-11 23:06:21,"Only worth a visit in the summer time, to take...",w8p4JW6-ERsbg8aB5wjEAg
7,r49iBfbnfoK7yt4rdsL_7g,0,2016-01-05 06:39:33,Hands down best AYCE in town! Love it!,CaFbX7topxdLpBJ_SNDfGQ
8,YCRCFBr6myPS49on2aS2pQ,0,2017-03-13 03:29:40,Sat in the drive thru this afternoon for 30 mi...,VLy6l8L72adD7iTBNqiPXw
9,RTBIiq-fJUXRsrzbL4MCqw,0,2013-02-08 20:55:15,Okay so I had a groupon for a nearby service c...,zrvk0-B91ahJRSB-UcAmIA
